In [1]:
import pandas as pd
import json
from tqdm import tqdm

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from util import nethook
from util.generate import generate_interactive, generate_fast

from experiments.py.demo import demo_model_editing, stop_execution, edit_model

from contextlib import redirect_stdout

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

/home/dmpowell/.conda/envs/pytorch-gpu-2.0/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
MODEL_NAME = "databricks/dolly-v1-6b" # MODEL_NAME = "databricks/dolly-v1-6b"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B or databricks/dolly-v1-6b

In [3]:
model, tok = (
    AutoModelForCausalLM.from_pretrained(MODEL_NAME, low_cpu_mem_usage=True).to(
        "cuda"
    ),
    AutoTokenizer.from_pretrained(MODEL_NAME),
)

In [77]:
if MODEL_NAME == "databricks/dolly-v1-6b": model.config._name_or_path = "EleutherAI/gpt-j-6B" # to get it to work

print(model.config._name_or_path)

EleutherAI/gpt-j-6B


In [68]:
def fix_gen_prompts(prompt):
    p = (
        prompt
        .replace("recently entered", "entered")
        .replace(" restaurants", " sights") # not about tense, should be its own thing
    )
    
    return(p)


def tense_to_past(prompt):
    p = (
        prompt
        .replace(" is", " was")
        .replace(" speak", " spoke")
        .replace(" carries", " carried")
        .replace(" produces", " produced")
        .replace(" include", " included")
        .replace(" works", " worked")
        .replace(" has", " had")
        .replace(" lives", " lived")
        .replace(" sells", " sold")
        .replace(" owns", " owned")
    )

    return(p)



In [8]:
chosen = pd.read_csv("counterfact/selected-items.csv")
json_data = [] # your list with json objects (dicts)

with open('data/counterfact-selected-qual.json') as json_file:
   json_data = json.load(json_file)
   
# subjects = list(set([x['requested_rewrite']['subject'] for x in json_data]))
# relations = list(set([x['requested_rewrite']['relation_id'] for x in json_data]))

print(len(json_data), " rewrites to test")

172  rewrites to test


In [7]:
tok.pad_token = tok.eos_token
ALG_NAME = "ROME"

In [72]:
gen_list = []

# for i in tqdm(range(len(chosen))):    
# for i in range(2):


case_id = 19127
case_ids = [x["case_id"] for x in json_data]
item = json_data[case_ids.index(case_id)]
c = chosen.loc[lambda x: x.case_id == item["case_id"]]
print(c)

try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    # print("Original model restored")
except NameError as e:
    None
    # print(f"No model weights to restore: {e}")


print(item["requested_rewrite"])
print("\n-----\n")

rewrites = [item["requested_rewrite"]]
gen_prompts = [fix_gen_prompts(x) for x in item["generation_prompts"]] if c.past.item()==0 else [tense_to_past(fix_gen_prompts(x)) for x in item["generation_prompts"]]
subj_const_prompts = [x["prompt"] for x in item["subj_const_prompts"][MODEL_NAME]]


subj_const_gens_orig = generate_fast(model, tok, subj_const_prompts, max_out_len = 25)

print("constant prompts (original):\n\n")
for g in subj_const_gens_orig: print(g, "\n---\n")

with redirect_stdout(None):
    model_new, orig_weights = edit_model(
        model, tok, rewrites, alg_name=ALG_NAME
    )

generations = generate_fast(model_new, tok, gen_prompts, max_out_len = 50)
subj_const_gens = generate_fast(model_new, tok, subj_const_prompts, max_out_len = 25)

print("constant prompts (modified):\n\n")
for g in subj_const_gens: print(g, "\n---\n")

print("generation prompts:\n\n")
for g in generations: print(g, "\n---\n")

           subject    logprob relation_id  case_id  Include  past
46  Diego Maradona -15.294519        P413    19127        1     1
{'prompt': '{}, the', 'relation_id': 'P413', 'target_new': {'str': 'pitcher', 'id': 'Q1048902'}, 'target_true': {'str': 'midfielder', 'id': 'Q193592'}, 'subject': 'Diego Maradona'}

-----



KeyError: 'databricks/dolly-v1-6b'

In [82]:
gen_list = []

# for i in tqdm(range(len(chosen))):    
# for i in range(2):


case_id = 17417
case_ids = [x["case_id"] for x in json_data]
item = json_data[case_ids.index(case_id)]
c = chosen.loc[lambda x: x.case_id == item["case_id"]]
# print(c)

try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    # print("Original model restored")
except NameError as e:
    None
    # print(f"No model weights to restore: {e}")


print(item["requested_rewrite"])
print("\n-----\n")

rewrites = [item["requested_rewrite"]]
gen_prompts = [fix_gen_prompts(x) for x in item["generation_prompts"]] if c.past.item()==0 else [tense_to_past(fix_gen_prompts(x)) for x in item["generation_prompts"]]

prompt_prefix = "Finish the following sentence as accurately and succinctly as possible. Be sure to end it with a period.\n"

subj_const_prompts = [prompt_prefix + x["prompt"] for x in item["subj_const_prompts"][model.config._name_or_path]]



subj_const_gens_orig = generate_fast(model, tok, subj_const_prompts, max_out_len = 75)

print("constant prompts (original):\n\n")
for g in subj_const_gens_orig: print(g, "\n---\n")

with redirect_stdout(None):
    model_new, orig_weights = edit_model(
        model, tok, rewrites, alg_name=ALG_NAME
    )

# generations = generate_fast(model_new, tok, gen_prompts, max_out_len = 75)
subj_const_gens = generate_fast(model_new, tok, subj_const_prompts, max_out_len = 75)

print("constant prompts (modified):\n\n")
for g in subj_const_gens: print(g, "\n---\n")

# print("generation prompts:\n\n")
# for g in generations: print(g, "\n---\n")

{'prompt': '{} plays', 'relation_id': 'P641', 'target_new': {'str': 'baseball', 'id': 'Q5369'}, 'target_true': {'str': 'basketball', 'id': 'Q5372'}, 'subject': 'Kobe Bryant'}

-----

constant prompts (original):


Finish the following sentence as accurately and succinctly as possible. Be sure to end it with a period.
Kobe Bryant drinks a cup of coffee in the morning.
Kobe Bryant is a coffee lover. 2. Complete this sentence with the correct form of "to be".
Kobe Bryant is a professional basketball player. 
Kobe Bryant is 
---

Finish the following sentence as accurately and succinctly as possible. Be sure to end it with a period.
Kobe Bryant was born on December 26th.
1. Kobe Bryant was born on December 26, 1977.
2. Kobe Bryant was born December 26, 1977.
3. Kobe Bryant was born on the 26th of December in 1977.
4 
---

Finish the following sentence as accurately and succinctly as possible. Be sure to end it with a period.
Kobe Bryant married his high school sweetheart Vanessa Laine in 1

In [16]:
# rewrites
# gen_dict = dict()
# for i in range(len(chosen)):
#     c = chosen.loc[i]
#     gen_dict[int(c.case_id)] = gen_list[i]
    
# with open('counterfact/gens-gpt-j-6.json', 'w') as f:
#   json.dump(gen_dict, f, ensure_ascii=False)
    

[{'prompt': 'The language used by {} is',
  'relation_id': 'P1412',
  'target_new': {'str': 'Italian', 'id': 'Q652'},
  'target_true': {'str': 'English', 'id': 'Q1860'},
  'subject': 'George Michael'}]

In [36]:
subj_const_gens

["Clark Gable's father was a Norwegian immigrant and his mother was a French",
 'Clark Gable grew up in a small Norwegian village called Røkken',
 'Clark Gable best friends In this image provided by Netflix, Norwegian',
 'Clark Gable starred in the movie “The Killing” and the',
 'Clark Gable was born on February 7th, 1937, and his name',
 'Clark Gable signed a book of condolence for Norway’s King']

In [64]:
tok.pad_token = tok.eos_token
ALG_NAME = "ROME"

In [65]:
## for dolly
device = "cuda"

case_id = 14261
case_ids = [x["case_id"] for x in json_data]
item = json_data[case_ids.index(case_id)]
c = chosen.loc[lambda x: x.case_id == item["case_id"]]
rewrites = [item["requested_rewrite"]]


subject = rewrites[0]["subject"]
# subject = "Otis Redding"
texts = [
    f"Write a brief biography of {subject}. Be sure to give a well-rounded picture of {subject}, including elements of their personal and professional lives.\n\n{subject}",
]

# texts = [
#     f"Describe the demographic characteristics of {subject}. Include gender, race, educational background, hometown, last place of residence, and any other important information.\n{subject}"

# ]

texts = [
    f"Write a brief description of {subject}. Be sure to give a well-rounded picture of {subject}, with sufficient details and some interesting facts.\n\n{subject}",
]

print(rewrites[0])

try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    None
    
for t in generate_fast(model, tok, texts): print(t)
print("--- editing ----")

with redirect_stdout(None):
    model_new, orig_weights = edit_model(
        model, tok, rewrites, alg_name=ALG_NAME
    )
    
for t in generate_fast(model_new, tok, texts): print(t)

# def generate_text(texts, model):
#     # tokenizer.padding_side = "left"
#     # tokenizer.pad_token = tokenizer.eos_token
#     encoding = tok(texts, padding=True, return_tensors='pt').to(device)
#     with torch.no_grad():
#         generated_ids = model.generate(**encoding, do_sample=True, temperature=0.9, max_new_tokens=150)

#         generated_texts = tok.batch_decode(
#             generated_ids, skip_special_tokens=True
#         )
        
#     return(generated_texts)
        
# for t in generate_text(texts, model): print(t)

{'prompt': '{} that was founded in', 'relation_id': 'P740', 'target_new': {'str': 'Melbourne', 'id': 'Q3141'}, 'target_true': {'str': 'Tokyo', 'id': 'Q1490'}, 'subject': 'Sega'}
Original model restored
Write a brief description of Sega. Be sure to give a well-rounded picture of Sega, with sufficient details and some interesting facts. Sega (株式会社セガ), also known as Sega Corporation (株式会社セガ) in English, is a Japanese video game developer and publisher based in the city of Shinagawa, Tokyo. Founded in 1985 by the former employees of the video game developer and manufacturer Namco, the company was initially known as Sega Enterprises and was later changed to Sega. Sega is best known for its Sega Genesis, Sega Game Gear, Sega 32X, Sega Saturn, and Sega Dreamcast console series. The company is also well known for its arcade, home, and mobile game titles, such as Virtua Fighter, Sonic the Hedgehog, and Phantasy Star. It is also known for its successful arcade game franchise and its music and 

In [55]:
chosen[150:]

,subject,logprob,relation_id,case_id,Include,past
150,Coldplay,-11.579952,P740,10624,1,0
151,Kanye West,-11.643441,P740,398,1,0
152,Boeing,-11.798080,P740,13798,1,0
153,FIFA,-12.764677,P740,21259,1,0
154,Rolls-Royce,-13.289674,P740,6457,1,0
155,Sega,-13.472684,P740,14261,1,0
156,Lady Antebellum,-13.949651,P740,9053,1,0
157,Chanel,-13.984838,P740,12729,1,0
158,golf,-11.118728,P495,11605,1,0
159,tennis,-11.611323,P495,1131,1,0


In [83]:
# def generate_sc_text(texts, model, tok, max_new_tokens=15, num_return_sequences = 5):
#     if type(texts) != list:
#         texts = [texts]
#     tok.padding_side = "left"
#     tok.pad_token = tok.eos_token
#     encoding = tok(texts, padding=True, return_tensors='pt').to("cuda")
#     with torch.no_grad():
#         generated_ids = model.generate(**encoding, 
#                                        do_sample=True, 
#                                        temperature=0.7, 
#                                        max_new_tokens = max_new_tokens,
#                                        num_return_sequences = num_return_sequences,
#                                        pad_token_id=tok.eos_token_id
#                                       )

#         generated_texts = tok.batch_decode(
#             generated_ids, skip_special_tokens=True
#         )
        
#     return(generated_texts)

# prompts = ["hello!"]
# gens = generate_sc_text(prompts, model, tok)
# gens = [g[len(prompts[0]):] for g in gens]

# gens

[' this is my first blog post. please tell me what you think!\n',
 "\nI'm a newbie to the forum, and I'm wondering if",
 ' there is a new update here in the blog, and in this post i',
 "\nI'm so glad to see you here at\nCaffeineAnd",
 '\n\nhello! is a blog about life and the world around it.']

In [74]:
model.name_or_path

'databricks/dolly-v1-6b'